In [7]:
PROJECT_ID = "padayon-ko-gemini"
LOCATION = "us-east5"
MODEL = "claude-3-5-sonnet@20240620"

ENDPOINT = f"https://{LOCATION}-aiplatform.googleapis.com"

In [21]:
from anthropic import AnthropicVertex

client = AnthropicVertex(region=LOCATION, project_id=PROJECT_ID)

with client.messages.stream(
    max_tokens=4096,
    messages=[
        {
            "role": "user",
            "content": """
            make function with an ipython display so i can interact with claude api calls within an ipynb with output printed to markdown with markdown rendered in vscode markdown ipynb. no need for API key im using GCP
            from anthropic import AnthropicVertex

client = AnthropicVertex(region=LOCATION, project_id=PROJECT_ID)

with client.messages.stream(
    max_tokens=4096,
    messages=[
        {
            "role": "user",
            "content": "with "memory" or ability to access history. i want to make a mini llm CHAT interface. make function with an ipython display so i can interact with claude api calls within an ipynb,
        }
    ],
    model=MODEL,
) as stream:
    for text in stream.text_stream:
        print(text, end="", flush=True)
            """,
        }
    ],
    model=MODEL,
) as stream:
    for text in stream.text_stream:
        print(text, end="", flush=True)

Here's a function that creates an interactive chat interface using IPython widgets to interact with the Claude API within a Jupyter notebook, with output rendered as Markdown in VSCode:

```python
import ipywidgets as widgets
from IPython.display import display, Markdown
from anthropic import AnthropicVertex

def claude_chat(location, project_id, model):
    client = AnthropicVertex(region=location, project_id=project_id)
    
    conversation_history = []

    def on_send(button):
        user_input = input_box.value
        conversation_history.append({"role": "user", "content": user_input})
        
        with client.messages.stream(
            max_tokens=4096,
            messages=conversation_history,
            model=model,
        ) as stream:
            response = ""
            for text in stream.text_stream:
                response += text
                output.value = f"**Claude:** {response}"
        
        conversation_history.append({"role": "assistant", "content

In [3]:
import ipywidgets as widgets
from IPython.display import display, Markdown, Code
from anthropic import AnthropicVertex
import re

PROJECT_ID = "padayon-ko-gemini"
LOCATION = "us-east5"
MODEL = "claude-3-5-sonnet@20240620"

ENDPOINT = f"https://{LOCATION}-aiplatform.googleapis.com"


def claude_chat(location, project_id, model):
    client = AnthropicVertex(region=location, project_id=project_id)
    conversation_history = []
    output_area = widgets.Output()

    def on_send(button):
        user_input = input_box.value
        conversation_history.append({"role": "user", "content": user_input})

        with output_area:
            display(Markdown(f"**You:** {user_input}"))

        with client.messages.stream(
            max_tokens=4096,
            messages=conversation_history,
            model=model,
        ) as stream:
            response = ""
            code_block = ""
            in_code_block = False

            for text in stream.text_stream:
                response += text

                if "```" in text:
                    in_code_block = not in_code_block
                    if not in_code_block and code_block:
                        with output_area:
                            lang = re.match(r"```(\w+)", code_block)
                            code = re.sub(r"```\w*\n?", "", code_block).strip()
                            display(
                                Code(
                                    data=code, language=lang.group(1) if lang else None
                                )
                            )
                        code_block = ""
                elif in_code_block:
                    code_block += text
                else:
                    with output_area:
                        display(Markdown(text), display_id=True)

            if code_block:
                with output_area:
                    lang = re.match(r"```(\w+)", code_block)
                    code = re.sub(r"```\w*\n?", "", code_block).strip()
                    display(Code(data=code, language=lang.group(1) if lang else None))

            conversation_history.append({"role": "assistant", "content": response})

        input_box.value = ""

    input_box = widgets.Text(placeholder="Type your message here...")
    send_button = widgets.Button(description="Send")
    send_button.on_click(on_send)

    display(widgets.VBox([input_box, send_button, output_area]))


# Usage:
claude_chat(LOCATION, PROJECT_ID, MODEL)